In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
import plotly.graph_objects as go
import lightgbm as lgbm  
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.impute import KNNImputer
from sklearn import linear_model
from sklearn.linear_model import HuberRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
import joblib

import warnings
warnings.filterwarnings('ignore')

In [53]:
df_train = pd.read_csv('/content/drive/MyDrive/tabular-playground-series-aug-2022/train.csv')# read data
df_test = pd.read_csv('/content/drive/MyDrive/tabular-playground-series-aug-2022/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/tabular-playground-series-aug-2022/sample_submission.csv')



In [54]:
Z = df_train['failure']
df_train.drop('failure',axis=1, inplace = True)#remove"failure"
Data = pd.concat([df_train, df_test])#

In [55]:
Data['missing_3'] = Data['measurement_3'].isnull().astype(np.int16)#Use astype() data type to convert to np.int16
Data['missing_5'] = Data['measurement_5'].isnull().astype(np.int16)

In [56]:
def Corr(d,x):
  #Correlation between two variables in data
  return  np.absolute(d.corr()[f'{x}']).sort_values(ascending=False)
def kk(t,c,m):
  train = t[c+[m]].dropna(how='any')
  test = t[(t[c].isnull().sum(axis=1)==0)&(t[m].isnull())]
  return train,test


In [57]:

Feature = [k for k in df_test.columns if k.startswith('measurement') or k=='loading']
#Treat words with "measurement" and "loading" as Feature
other = [other for other in df_test.columns if other.startswith('attribute')or other=='id']+ ['loading','product_code','missing_3','missing_5']
s = []
n =[]
for x in range(3,18):  
    corr =Corr(Data.drop(other, axis=1),f'measurement_{x}')
    s.append(np.round(np.sum(corr[1:5]),5)) 
    # add the 5 first lines of the correlation 
    n.append(f'measurement_{x}')
show = pd.DataFrame()
show['columns'] = n
show['correlation'] = s #most correlated
show = show.sort_values(by = 'correlation',ascending=False).reset_index(drop = True)


In [58]:
FullDict ={}
for i in range(8):
    next = show.iloc[i,0] 
    # select the next best correlated column 
    Fill ={}
    for x in Data.product_code.unique() : 
      NextDic = {}
      corr = Corr(Data[Data.product_code == x],next)
      NextDic[next] = corr[1:5].index.tolist()
      Fill[x] = NextDic[next]
    FullDict[next] =Fill


In [59]:
Feature = [k for k in Data.columns if k.startswith('measurement') or k=='loading']
nulls = [col for col in df_train.columns if df_train[col].isnull().sum()!=0]
for i in Data.product_code.unique():
    TotalNa_filled = 0
    for measure_col in list(FullDict.keys()):
        model = HuberRegressor(epsilon=1.35, max_iter = 400)
        t = Data[Data.product_code==i]
        column = FullDict[measure_col][i]
        ttrain,ttest= kk(t,column,measure_col)
        model.fit(ttrain[column], ttrain[measure_col])
        Data.loc[(Data.product_code==i)&(Data[column].isnull().sum(axis=1)==0)&(Data[measure_col].isnull()),measure_col] = model.predict(ttest[column])
        TotalNa_filled += len(ttest)
        
    # others NA columns:
    NA = Data.loc[Data["product_code"] == i,nulls].isnull().sum().sum()
    model1 = KNNImputer(n_neighbors=3)
    Data.loc[Data.product_code==i,Feature] = model1.fit_transform(Data.loc[Data.product_code==i, Feature])
    joblib.dump(model1, 'KNN_model')

In [60]:
def scale(train_data, val_data, test_data, feats):
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(train_data[feats])
    scaled_val = scaler.transform(val_data[feats])
    scaled_test = scaler.transform(test_data[feats])
    new_train = train_data.copy()
    new_val = val_data.copy()
    new_test = test_data.copy()
    new_train[feats] = scaled_train
    new_val[feats] = scaled_val
    new_test[feats] = scaled_test
    return new_train, new_val, new_test

In [61]:
LE= LabelEncoder()
Data['attribute_1'] = LE.fit_transform(Data['attribute_1'])

In [62]:
! pip install feature_engine
from feature_engine.encoding import WoEEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
df_train = Data.iloc[:df_train.shape[0],:]
df_test = Data.iloc[df_train.shape[0]:,:]

G = df_train.product_code
X = df_train
woe = WoEEncoder(variables=['attribute_0'])
woe.fit(X, Z)
w = woe.transform(X)
test = woe.transform(df_test)

In [66]:
class Train3_Valid2():
    def __init__(self):
        self.name = "Splitter with 2 product code in validation set"
        
    def split(self, X: pd.DataFrame, _y=None, _groups=None):
        indices = list(X.groupby("product_code").indices.values())
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                yield [np.concatenate([ix for k, ix in enumerate(indices) if k not in [i, j]]),
                    np.concatenate([ix for k, ix in enumerate(indices) if k in [i, j]])]

In [67]:

select = ['loading','attribute_0','measurement_17','measurement_0',
          'measurement_1','measurement_2','missing_3','missing_5',]

In [69]:
import pickle
with open('/content/drive/MyDrive/tabular-playground-series-aug-2022/x.pickle', 'rb') as f:
    Model = pickle.load(f)
final = np.zeros(len(test))
cv = Train3_Valid2()
kf = GroupKFold(n_splits=5)
SPLIT = 5
for fold_idx, (train_idx, val_idx) in enumerate(kf.split( w, Z, G)):# = train.product_code 
    x_train, x_val = w.iloc[train_idx], w.iloc[val_idx]
    x_train, x_val, x_test = scale(x_train, x_val, test, select)
    final += Model.predict_proba(x_test[select])[:, 1] / SPLIT
submission['failure'] = final
submission.to_csv(f"./109550106.csv", index=False)
submission
   

,id,failure
0,26570,0.201389
1,26571,0.193210
2,26572,0.198365
3,26573,0.200756
4,26574,0.240616
...,...,...
20770,47340,0.216162
20771,47341,0.188960
20772,47342,0.189176
20773,47343,0.212924
